In [24]:
import os
from PIL import Image
import re
from tqdm import tqdm 
from spellchecker import SpellChecker
import re
import pytesseract
from transformers import AutoTokenizer, T5ForConditionalGeneration
import os

# Set TOKENIZERS_PARALLELISM to true
os.environ["TOKENIZERS_PARALLELISM"] = "true"

spell = SpellChecker()

tokenizer = AutoTokenizer.from_pretrained("grammarly/coedit-xl")
model = T5ForConditionalGeneration.from_pretrained("grammarly/coedit-xl")

spell = SpellChecker()

def correct_grammar(text):
    # Encode the text input
    text = "Fix grammatical errors in this sentence: " + text
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    # Generate corrected text output
    outputs = model.generate(input_ids, max_length=8192)
    # Decode and return the corrected text
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

def assess_ocr_quality(text):
    # Number Heavy Check
    num_ratio = sum(c.isdigit() for c in text) / len(text) if text else 0
    if num_ratio > 0.5:
        return 'number_heavy'

    # Basic Length Check
    if len(text) < 20:  # arbitrary minimum length
        return 'unusable'

    # Spell Checking and Basic Grammar Analysis
    words = text.split()
    misspelled = spell.unknown(words)
    if len(misspelled) > 0.4 * len(words):  # if more than 40% of words are misspelled
        return 'unusable'

    # Simple Coherence Check (can be improved with NLP techniques)
    # Example: Check for frequent occurrences of nonsensical character combinations
    if re.search(r'[^\w\s]', text):  # regex for non-word, non-space characters
        return 'okay'

    # If none of the above, classify as good
    return 'good'

source_dir = "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/images"
target_dirs = {
    'good': "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/image_ocrs/good_ocr",
    'okay': "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/image_ocrs/okay_ocr",
    'unusable': "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/image_ocrs/unusable_ocr",
    'number_heavy': "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/image_ocrs/number_heavy_ocrs"
}
if not os.path.exists(source_dir):
    print("Source directory does not exist.")
    exit(1)

for dir in target_dirs.values():
    if not os.path.exists(dir):
        os.makedirs(dir)

# Get list of image files
image_files = [f for f in os.listdir(source_dir) if f.lower().endswith((".jpg", ".png"))]

# Process each file with a progress bar
for filename in tqdm(image_files, desc="Processing images"):
    image_path = os.path.join(source_dir, filename)
    try:
        text = pytesseract.image_to_string(Image.open(image_path))
        text = correct_grammar(text)
        quality = assess_ocr_quality(text)
        with open(os.path.join(target_dirs[quality], filename + ".txt"), "w") as text_file:
            text_file.write(text)
    except Exception as e:
        print(f"Error processing {filename}: {e}")



Processing images:   0%|                                | 0/441 [00:00<?, ?it/s]

Processing images:   0%|                      | 1/441 [00:50<6:09:05, 50.33s/it]

Processing images:   0%|                      | 2/441 [01:15<4:20:42, 35.63s/it]

Processing images:   1%|▏                     | 3/441 [01:49<4:14:20, 34.84s/it]

Processing images:   1%|▏                     | 4/441 [02:09<3:29:50, 28.81s/it]Token indices sequence length is longer than the specified maximum sequence length for this model (600 > 512). Running this sequence through the model will result in indexing errors


Processing images:   1%|▏                     | 5/441 [03:01<4:30:12, 37.19s/it]

Processing images:   1%|▎                     | 6/441 [03:46<4:49:34, 39.94s/it]

Processing images:   2%|▎                     | 7/441 [04:32<5:03:12, 41.92s/it]

Processing images:   2%|▍                     | 8/441 [06:22<7:38:59, 63.60s/it]

Processing images:   2%|▍                     | 9/441 [07:16<7:16:00, 60.56s/i

KeyboardInterrupt: 